In [1]:
# pip install -q pyyaml h5py  # Required to save models in HDF5 format

import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.3.0


In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

In [3]:
# Define a simple sequential model
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512, activation='relu', input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10)
    ])

    model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.metrics.SparseCategoricalAccuracy()])

    return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [4]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])  # Pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

Epoch 1/10
29/32 [==========================>...] - ETA: 0s - loss: 1.2578 - sparse_categorical_accuracy: 0.6358
Epoch 00001: saving model to training_1\cp.ckpt
32/32 [==============================] - 1s 22ms/step - loss: 1.2132 - sparse_categorical_accuracy: 0.6500 - val_loss: 0.7346 - val_sparse_categorical_accuracy: 0.7780
Epoch 2/10
30/32 [===========================>..] - ETA: 0s - loss: 0.4270 - sparse_categorical_accuracy: 0.8844
Epoch 00002: saving model to training_1\cp.ckpt
32/32 [==============================] - 0s 12ms/step - loss: 0.4278 - sparse_categorical_accuracy: 0.8820 - val_loss: 0.5217 - val_sparse_categorical_accuracy: 0.8400
Epoch 3/10
25/32 [======================>.......] - ETA: 0s - loss: 0.3030 - sparse_categorical_accuracy: 0.9237
Epoch 00003: saving model to training_1\cp.ckpt
32/32 [==============================] - 0s 14ms/step - loss: 0.2921 - sparse_categorical_accuracy: 0.9250 - val_loss: 0.5244 - val_sparse_categorical_accuracy: 0.8300
Epoch 4/10
31

In [5]:
ls {checkpoint_dir}

 Volume in drive C is OS
 Volume Serial Number is 88CB-3D69

 Directory of C:\Users\Noam\Desktop\AI\repos\tf-notebooks\beginner-ml-basics-with-keras\training_1

09/22/2020  10:40 PM    <DIR>          .
09/22/2020  10:40 PM    <DIR>          ..
09/22/2020  10:40 PM                71 checkpoint
09/22/2020  10:40 PM         4,886,673 cp.ckpt.data-00000-of-00001
09/22/2020  10:40 PM             1,222 cp.ckpt.index
               3 File(s)      4,887,966 bytes
               2 Dir(s)  54,566,240,256 bytes free


In [6]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 2.3234 - sparse_categorical_accuracy: 0.0870
Untrained model, accuracy:  8.70%
32/32 - 0s - loss: 0.4199 - sparse_categorical_accuracy: 0.8630
Restored model, accuracy: 86.30%


In [7]:
# more options in callback
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=5)

# Create a new model instance
model = create_model()

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(train_images, 
          train_labels,
          epochs=50, 
          callbacks=[cp_callback],
          validation_data=(test_images,test_labels),
          verbose=0)


Epoch 00005: saving model to training_2\cp-0005.ckpt

Epoch 00010: saving model to training_2\cp-0010.ckpt

Epoch 00015: saving model to training_2\cp-0015.ckpt

Epoch 00020: saving model to training_2\cp-0020.ckpt

Epoch 00025: saving model to training_2\cp-0025.ckpt

Epoch 00030: saving model to training_2\cp-0030.ckpt

Epoch 00035: saving model to training_2\cp-0035.ckpt

Epoch 00040: saving model to training_2\cp-0040.ckpt

Epoch 00045: saving model to training_2\cp-0045.ckpt

Epoch 00050: saving model to training_2\cp-0050.ckpt


In [8]:
ls {checkpoint_dir}

 Volume in drive C is OS
 Volume Serial Number is 88CB-3D69

 Directory of C:\Users\Noam\Desktop\AI\repos\tf-notebooks\beginner-ml-basics-with-keras\training_2

09/22/2020  10:40 PM    <DIR>          .
09/22/2020  10:40 PM    <DIR>          ..
09/22/2020  10:40 PM                81 checkpoint
09/22/2020  10:40 PM         1,628,726 cp-0000.ckpt.data-00000-of-00001
09/22/2020  10:40 PM               402 cp-0000.ckpt.index
09/22/2020  10:40 PM         4,886,685 cp-0005.ckpt.data-00000-of-00001
09/22/2020  10:40 PM             1,222 cp-0005.ckpt.index
09/22/2020  10:40 PM         4,886,685 cp-0010.ckpt.data-00000-of-00001
09/22/2020  10:40 PM             1,222 cp-0010.ckpt.index
09/22/2020  10:40 PM         4,886,685 cp-0015.ckpt.data-00000-of-00001
09/22/2020  10:40 PM             1,222 cp-0015.ckpt.index
09/22/2020  10:40 PM         4,886,685 cp-0020.ckpt.data-00000-of-00001
09/22/2020  10:40 PM             1,222 cp-0020.ckpt.index
09/22/2020  10:40 PM         4,886,685 cp-0025.ckpt.data

In [9]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2\\cp-0050.ckpt'

In [10]:
# Create a new model instance
model = create_model()

# Load the previously saved weights
model.load_weights(latest)

# Re-evaluate the model
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4777 - sparse_categorical_accuracy: 0.8810
Restored model, accuracy: 88.10%


### Manually save weights

In [11]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Create a new model instance
model = create_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4777 - sparse_categorical_accuracy: 0.8810
Restored model, accuracy: 88.10%


### save the entire model

In [12]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model') 

Epoch 1/5
32/32 [==============================] - 0s 5ms/step - loss: 1.1475 - sparse_categorical_accuracy: 0.6810
Epoch 2/5
32/32 [==============================] - 0s 6ms/step - loss: 0.4203 - sparse_categorical_accuracy: 0.8730
Epoch 3/5
32/32 [==============================] - 0s 5ms/step - loss: 0.2669 - sparse_categorical_accuracy: 0.9340
Epoch 4/5
32/32 [==============================] - 0s 6ms/step - loss: 0.2112 - sparse_categorical_accuracy: 0.9520A: 0s - loss: 0.2028 - sparse_categorical_accuracy: 0.9
Epoch 5/5
32/32 [==============================] - 0s 6ms/step - loss: 0.1481 - sparse_categorical_accuracy: 0.9650
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: saved_model/my_model\assets


In [13]:
# my_model directory
!ls saved_model

# Contains an assets folder, saved_model.pb, and variables folder.
!ls saved_model/my_model

my_model
assets
saved_model.pb
variables


In [14]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()

# Evaluate the restored model
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

print(new_model.predict(test_images).shape)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________
32/32 - 0s - loss: 0.4115 - sparse_categorical_accuracy: 0.8640
Restored model, accuracy: 86.40%
(1000, 10)


### HD5Format

In [16]:
# Recreate the exact same model, including its weights and the optimizer
# new_model = tf.keras.models.load_model('my_model.h5')

# # Show the model architecture
# new_model.summary()

# loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
# print('Restored model, accuracy: {:5.2f}%'.format(100*acc))